In [1]:
import json
import os
import networkx as nx
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

In [2]:
SRC_OBJ_FILE = "../MP1-assignment-PyCharm/VisualGenome-data-SMALL/objects.json"
SRC_REL_FILE = "../MP1-assignment-PyCharm/VisualGenome-data-SMALL/relationships.json"
SRC_OBJ_ALIAS_FILE = "../MP1-assignment-PyCharm/VisualGenome-data/object_alias.txt"
SRC_REL_ALIAS_FILE = "../MP1-assignment-PyCharm/VisualGenome-data/relationship_alias.txt"
DEBUG_ = False

In [3]:
def construct_object_alias_dict_map():
    alias_data = open(SRC_OBJ_ALIAS_FILE, 'r')
    output_map = {}
    for alias_line in alias_data:
        tmp_list = alias_line.split(",")
        tmp_src = tmp_list[0]
        tmp_targets = tmp_list[1:]
        for tmp_target in tmp_targets:
            output_map[tmp_target] = tmp_src
    return output_map

In [4]:
def construct_relationships_alias_dict_map():
    alias_data = open(SRC_REL_ALIAS_FILE, 'r')
    output_map = {}
    for alias_line in alias_data:
        tmp_list = alias_line.split(",")
        tmp_src = tmp_list[0]
        tmp_targets = tmp_list[1:]
        for tmp_target in tmp_targets:
            output_map[tmp_target] = tmp_src
    return output_map

In [5]:
G = nx.Graph()

object_alias_map = construct_object_alias_dict_map()
rel_alias_map = construct_relationships_alias_dict_map()

data = json.load(open(SRC_REL_FILE, 'r'))  # 10,000
n_data = len(data)
print("No. images : ", n_data)
for image_idx, image in enumerate(data):
    tmp_img_rel = image['relationships']
    for rel_idx, rel in enumerate(tmp_img_rel):
        # primary types
        tmp_rel_id = rel['relationship_id']
        tmp_synsets_detailed_rel = rel['synsets']
        tmp_predicate = rel['predicate']
        tmp_object = rel['object']
        tmp_subject = rel['subject']

        # object type
        tmp_object_id = tmp_object['object_id']
        tmp_object_synsets_detailed_obj = tmp_object['synsets']
        # tmp_object_merged_object_ids = tmp_object.get('merged_object_ids', "")  # maybe linked mention ???
        # tmp_object_names = tmp_object['names']  # difference between 'name' and 'names'
        if tmp_object.get('names', "") != "":
            tmp_object_names = tmp_object['names']
        else:
            tmp_object_names = tmp_object['name']
        if (type(tmp_object_names) == list) and (len(tmp_object_names) > 1):
            print("object found as list with len : ", len(tmp_object_names))
        elif type(tmp_object_names) == list:
            tmp_object_name = tmp_object_names[0]

        # subject type
        tmp_subject_id = tmp_subject['object_id']
        tmp_subject_synsets_detailed_obj = tmp_subject['synsets']  # why sometimes synsets are empty list
        if tmp_subject.get('name', "") != "":
            tmp_subject_name = tmp_subject['name']
        else:
            tmp_subject_name = tmp_subject['names']
        if (type(tmp_subject_name) == list) and (len(tmp_subject_name) > 1):
            print("subject found as list with len : ", len(tmp_subject_name))
        elif type(tmp_subject_name) == list:
            tmp_subject_name = tmp_subject_name[0]

        # Adding nodes to graph
        if tmp_subject_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_subject_name = object_alias_map[tmp_subject_name]
        if tmp_object_name in object_alias_map.keys():
            # print("Found synonym for object name")
            tmp_object_name = object_alias_map[tmp_object_name]
        G.add_node(tmp_subject_name, id=tmp_subject_id, synsets=tmp_subject_synsets_detailed_obj)
        G.add_node(tmp_object_name, id=tmp_object_id, synsets=tmp_object_synsets_detailed_obj)
        # Adding edges to graph : this should actually be a DI-GRAPH
        if tmp_predicate in rel_alias_map.keys():
            tmp_predicate = rel_alias_map[tmp_predicate]
        G.add_edge(tmp_subject_name, tmp_object_name, id=tmp_rel_id, synsets=tmp_synsets_detailed_rel,
                       predicate=tmp_predicate)
        
    if image_idx % 1000 == 0:
        print("finished loading image : {} / {}".format(image_idx + 1, n_data))
        
#     if image_idx == 10:
#         break

#         if rel_idx == 0:
#             # nx.draw(G, with_labels=True, font_weight='bold')
#             nx.draw_networkx(G, pos=nx.spring_layout(G), node_size=100, node_color='r', node_shape='o', alpha=0.5,
#                              linewidths=1.0, width=1.0,
#                              )
#             # plt.savefig("sample_image_SceneGraph.svg")
#             plt.show()

No. images :  10000
finished loading image : 1 / 10000
finished loading image : 1001 / 10000
finished loading image : 2001 / 10000
finished loading image : 3001 / 10000
finished loading image : 4001 / 10000
finished loading image : 5001 / 10000
finished loading image : 6001 / 10000
finished loading image : 7001 / 10000
finished loading image : 8001 / 10000
finished loading image : 9001 / 10000


# 2. Analysis

In [6]:
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.

In [7]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 7747
Number of edges: 48965
Average degree:  12.6410


In [8]:
density = nx.density(G)
print("Network density:", density)

"""
This is simply the ratio of actual edges in the network to all possible edges in the network. 
Network density gives you a quick sense of how closely knit your network is.
"""

Network density: 0.0016319419482603913


'\nThis is simply the ratio of actual edges in the network to all possible edges in the network. \nNetwork density gives you a quick sense of how closely knit your network is.\n'

In [10]:
G.nodes()

NodeView(('shade', 'sidewalk', 'man', 'shoes', 'car', 'sign', 'tree trunk', 'parking meter', 'bike', 'van', 'shirt', 'lamp post', 'street', 'tree', 'building', 'crosswalk', 'walk sign', 'keyboard', 'computer tower', 'filing cabinet', 'mouse', 'wireless phone', 'telephone', 'pluged', 'outlet', 'girl', 'computer', 'plug', 'pen', 'cable', 'wall', 'bag', 'woman', 'hair', 'partition', 'picture', 'photo', 'chain', 'chair', 'carpet', 'pillow', 'teddy bear', 'animal', 'table top', 'futon', 'door', 'doll', 'lamp', 'book', 'printer', 'railing', 'computer monitor', 'fax machine', 'cork-board', 'paper', 'air conditioner', 'shelf', 'room', 'clothes', 'heater', 'apple', 'sports bottle', 'milk', 'glass', 'top', 'water bottle', 'drawer', 'fork', 'liquid', 'bar stool', 'bowl', 'pan', 'food', 'container', 'leg', 'crumb', 'bottle', 'juice bottle', 'sticky note', 'white board', 'headphones', 'papers', 'cpu', 'bracelet', 'wrist pad', 'cd player', 'cereal', 'monitor', 'mug', 'laptop', 'tape dispenser', 'cal

In [12]:
fell_whitehead_path = nx.shortest_path(G, source="picture", target="paper")

print("Shortest path between picture and paper:", fell_whitehead_path)
print("Length of that path:", len(fell_whitehead_path)-1)

Shortest path between picture and paper: ['picture', 'coffee mug', 'keyboard', 'paper']


In [14]:
### Diameter

In [9]:
# If your Graph has more than one component, this will return False:
print(nx.is_connected(G))

False


In [11]:
# Next, use nx.connected_components to get the list of components,
# then use the max() command to find the largest one:
components = nx.connected_components(G)
largest_component = max(components, key=len)
# print(components)
# print(largest_component)

In [12]:
# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.
#

subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

KeyboardInterrupt: 

In [18]:
### triadic closure

In [13]:
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

"""
Transitivity is the ratio of all triangles over all possible triangles.
So transitivity, like density, expresses how interconnected a graph is in terms of a 
    ratio of actual over possible connections
Transitivity allows you a way of thinking about all the relationships in your graph that may exist but currently do not.

"""

Triadic closure: 0.1322274135806441


'\nTransitivity is the ratio of all triangles over all possible triangles.\nSo transitivity, like density, expresses how interconnected a graph is in terms of a \n    ratio of actual over possible connections\nTransitivity allows you a way of thinking about all the relationships in your graph that may exist but currently do not.\n\n'

In [21]:
# densite < triadic_closure:
#     reason:
#         Because the graph is not very dense, there are fewer possible triangles to begin with, 
#         which may result in slightly higher transitivity.


In [22]:
### Centrality - Which nodes are the most important?

In [14]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

In [15]:
print(G.nodes['bracelet'])

{'id': 4460480, 'synsets': [], 'degree': 48}


In [16]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [17]:
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

Top 20 nodes by degree:
('man', 1258)
('building', 1107)
('window', 906)
('person', 821)
('road', 805)
('car', 791)
('woman', 710)
('tree', 688)
('sign', 589)
('table', 581)
('light', 567)
('ground', 561)
('chair', 552)
('wall', 546)
('a', 471)
('shirt', 467)
('leaf', 459)
('door', 448)
('people', 445)
('desk', 423)


In [18]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

KeyboardInterrupt: 

In [28]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

Top 20 nodes by betweenness centrality:
('coffee mug', 0.4815058290799493)
('railing', 0.3161342022095898)
('keyboard', 0.2355541471565764)
('laptop', 0.19057144303129372)
('man', 0.18584924401829087)
('shoes', 0.1014678684149872)
('street', 0.07868810833752637)
('computer', 0.07300222206585866)
('mouse', 0.061438057150878064)
('telephone', 0.04785840081930871)
('table', 0.0360768019413578)
('plant', 0.03439118271179342)
('sun', 0.03273851465138916)
('hair', 0.031999936099699165)
('chair', 0.029353420594768972)
('cable', 0.028971705331871787)
('computer tower', 0.023577808841622968)
('sidewalk', 0.015267175572519083)
('girl', 0.015267175572519083)
('cup', 0.012770365087303474)


In [29]:
sorted_eigenvector = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by eigenvector centrality:")
for b in sorted_eigenvector[:20]:
    print(b)

Top 20 nodes by eigenvector centrality:
('coffee mug', 0.47073664518445407)
('railing', 0.35685427063487696)
('laptop', 0.30361117926051573)
('keyboard', 0.2905663521341458)
('computer', 0.17713248226199577)
('cable', 0.17212134480128086)
('mouse', 0.16073463527867052)
('telephone', 0.13426105082219264)
('chair', 0.1316509343852216)
('water bottle', 0.11683991659427549)
('headphones', 0.11683991659427549)
('book', 0.11549227289932651)
('computer tower', 0.11413818897902082)
('table', 0.11373240567273592)
('woman', 0.10520474786855451)
('bag', 0.09826145831196297)
('monitor', 0.09722782129527271)
('cup', 0.09613922108633503)
('pen', 0.09348842889578991)
('picture', 0.08695482131354534)


In [30]:
# You’ll notice that many, but not all, 
# of the nodes that have high degree also have high betweenness centrality.

In [33]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "\t| Betweenness Centrality:", tb[1], "\t| Degree:", degree)

Name: coffee mug 	| Betweenness Centrality: 0.4815058290799493 	| Degree: 50
Name: railing 	| Betweenness Centrality: 0.3161342022095898 	| Degree: 43
Name: keyboard 	| Betweenness Centrality: 0.2355541471565764 	| Degree: 29
Name: laptop 	| Betweenness Centrality: 0.19057144303129372 	| Degree: 29
Name: man 	| Betweenness Centrality: 0.18584924401829087 	| Degree: 4
Name: shoes 	| Betweenness Centrality: 0.1014678684149872 	| Degree: 9
Name: street 	| Betweenness Centrality: 0.07868810833752637 	| Degree: 9
Name: computer 	| Betweenness Centrality: 0.07300222206585866 	| Degree: 13
Name: mouse 	| Betweenness Centrality: 0.061438057150878064 	| Degree: 8
Name: telephone 	| Betweenness Centrality: 0.04785840081930871 	| Degree: 9
Name: table 	| Betweenness Centrality: 0.0360768019413578 	| Degree: 12
Name: plant 	| Betweenness Centrality: 0.03439118271179342 	| Degree: 6
Name: sun 	| Betweenness Centrality: 0.03273851465138916 	| Degree: 6
Name: hair 	| Betweenness Centrality: 0.0319999

In [34]:
### Community detection with modularity

In [35]:
# what the subgroups or communities are within the larger social structure.
# Modularity is a measure of relative density in your network: a community 
#     (called a module or modularity class) has high density relative to other nodes 
#     within its module but low density with those outside. 
# Modularity gives you an overall score of how fractious your network is, and that 
#     score can be used to partition the network and return the individual communities
# Very dense networks are often more difficult to split into sensible partition

In [36]:
# Degree Rank Code i

# e some built-in approaches to community detection (like minimum cut, but modularity is not included with NetworkX.

In [19]:
communities = community.greedy_modularity_communities(G)

KeyboardInterrupt: 

In [ ]:
modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

In [ ]:
# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

In [40]:
for i,c in enumerate(communities): # Loop through the list of communities
    if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':', list(c)) # Print out the classes and their members

Class 0: ['cd player', 'apple', 'glass', 'crumb', 'water bottle', 'bracelet', 'liquid', 'bottle', 'railing', 'clothes', 'sticky note', 'bar stool', 'cpu', 'heater', 'headphones', 'food', 'milk', 'drawer', 'top', 'leg', 'sports bottle', 'fork', 'wrist pad', 'papers', 'pan']
Class 1: ['tube', 'jacket', 'light', 'guy', 'baby', 'wires', 'files', 'ostirch', 'window', 'board', 'charger', 'dish', 'coat', 'folder', 'blinds', 'corkboard', 'bulletin board', 'tower', 'panes', 'bin', 'coffee mug', 'posters', 'bookshelf', 'mouse pad']
Class 2: ['cork-board', 'air conditioner', 'printer', 'doll', 'door', 'teddy bear', 'fax machine', 'animal', 'shelf', 'futon', 'keyboard', 'carpet', 'computer tower', 'paper', 'walk sign', 'table top', 'pillow', 'room', 'book', 'lamp', 'computer monitor']
Class 3: ['calculator', 'woman', 'picture', 'boy', 'pen', 'drawers', 'juice bottle', 'photograph', 'tape dispenser', 'container', 'white board', 'cd', 'partition', 'hand', 'laptop', 'stapler', 'cable', 'cereal', 'pho

# Node degree based analysis

In [20]:
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

Top 20 nodes by degree:
('man', 1258)
('building', 1107)
('window', 906)
('person', 821)
('road', 805)
('car', 791)
('woman', 710)
('tree', 688)
('sign', 589)
('table', 581)
('light', 567)
('ground', 561)
('chair', 552)
('wall', 546)
('a', 471)
('shirt', 467)
('leaf', 459)
('door', 448)
('people', 445)
('desk', 423)


In [24]:
node_MAN_neighbors = list(G.nodes('man'))
node_MAN_neighbors[:20]

[('shade', None),
 ('sidewalk', None),
 ('man', None),
 ('shoes', None),
 ('car', None),
 ('sign', None),
 ('tree trunk', None),
 ('parking meter', None),
 ('bike', None),
 ('van', None),
 ('shirt', None),
 ('lamp post', None),
 ('street', None),
 ('tree', None),
 ('building', None),
 ('crosswalk', None),
 ('walk sign', None),
 ('keyboard', None),
 ('computer tower', None),
 ('filing cabinet', None)]

In [27]:
top2_node_path = nx.shortest_path(G, source="man", target="building")

print("Shortest path between 'man' and 'building':", top2_node_path)
print("Length of that path:", len(top2_node_path)-1)

Shortest path between 'man' and 'building': ['man', 'building']
Length of that path: 1


# Degree centrality

In [30]:
def find_nodes_with_highest_deg_cent(G):

    # Compute the degree centrality of G: deg_cent
    deg_cent = nx.degree_centrality(G)
    # Compute the maximum degree centrality: max_dc
    max_1_dc = max(list(deg_cent.values()))
    max_2_dc = list(sorted(deg_cent.values()))[-2]
    max_3_dc = list(sorted(deg_cent.values()))[-3]

    maxnode1 = set()
    maxnode2 = set()
    maxnode3 = set()

    # Iterate over the degree centrality dictionary
    for k, v in deg_cent.items():

        # Check if the current value has the maximum degree centrality
        if v == max_1_dc:

            # Add the current node to the set of nodes
            maxnode1.add(k)
        if v == max_2_dc:

            # Add the current node to the set of nodes
            maxnode2.add(k)
        if v == max_3_dc:

            # Add the current node to the set of nodes
            maxnode3.add(k)

    return maxnode1,maxnode2,maxnode3


top_deg_dc,top2_deg_dc,top3_deg_dc = find_nodes_with_highest_deg_cent(G)
print(top_deg_dc,top2_deg_dc,top3_deg_dc)

{'man'} {'building'} {'window'}


# Clustering coefficient

In [33]:
print(nx.clustering(G, 'man'))
print(nx.clustering(G, 'building'))
print(nx.clustering(G, 'window'))

0.032950998553556474
0.03674831136467965
0.05503777967247817


In [34]:
nx.average_clustering(G)

0.19870084229129634

# BFS Search

In [35]:
S = nx.bfs_tree(G, 'keyboard')
nx.draw_networkx(S)

KeyboardInterrupt: 

# Eccentricity

In [38]:
print(nx.eccentricity(G, 'town'))

NetworkXError: Found infinite path length because the graph is not connected